In [39]:
import pandas as pd
import xlrd
import re
import numpy as np
import sys
from IPython.display import display

from cookie import Cookie

In [2]:
# Creating cookies and other ingredient objects. Supplying recipe and desired (i.e., 
# amount of cookies I want) yield for class attributes
thin_and_crispy_cookie = Cookie(25, 122, 30)
brown_snickerdoodle_cookie = Cookie(26,78, 56)
cold_brew_cookie = Cookie(29, 87, 47)
toffee = Cookie(280, 280, 120)

In [3]:
# a function to remove misc measurement label
def remove_measurements(weight):
    # in case any empty values are given just return (aka go to the next row/value)
    if not isinstance(weight, str):
        weight = str(weight)
    # remove any terms of measurement (aka any letters) and 
    # return float value
    return float(re.sub('[a-zA-Z ]*', '', weight))

# The three below functions will perform the following:
# Cost = (weight-of-recipe-yield/base-weight-of-ingredient)*price-of-ingredient
# def calculate_quotient(row):
#     # a function that will return the quotient of weights (recipe/base)
#     weight_base = remove_measurements(row['Weight/Amount_base'])
#     weight_recipe = remove_measurements(row['Weight/Amount_recipe'])
#     quotient = weight_recipe/weight_base 
#     return quotient

# def calculate_cost_for_original_recipe(row):
#     # a function that will multiply cost to returned quotient
#     quotient = calculate_quotient(row)
#     cost = round(quotient * row['Cost_base'], 2)
#     return cost

def calculate_cost_for_desired_yield(row, yield_difference):
    weight_of_item = remove_measurements(row['Weight/Amount_recipe'])
    units_required = weight_of_item * yield_difference
    return units_required

In [24]:
def ingest_excel(base_df, recipe_workbook):
    '''
    Merge the base ingredient excel spreadsheet with the recipe worksheet.
    Changes suffix to give greater detail to what matched columns belong to
    which spreadsheet
    '''
    base_df['Brand'] = base_df['Brand'].str.lower()
    base_df['Ingredient']= base_df['Ingredient'].str.lower()
    recipe_workbook['Brand'] = recipe_workbook['Brand'].str.lower()
    recipe_workbook['Ingredient'] = recipe_workbook['Ingredient'].str.lower()
    merged_df = base_df.merge(recipe_workbook, on=['Brand', 'Ingredient'], suffixes=('_base', '_recipe'))
    return merged_df

# dataframes for ingredients
base_df = pd.read_excel('Cookie Base Ingredient.xlsx')

# Working on Base Spreadsheet

In [25]:
# add a column for per unit cost of base ingredients
def price_per_unit(row):
    if row.isnull().any():
        price = 0
    else:
        weight_of_item = remove_measurements(row['Weight/Amount'])
        price = round(float(row['Cost'])/weight_of_item, 3)
    return price

price_per_unit_column = 'Per_Unit_Price'
base_df[price_per_unit_column] = base_df.apply(price_per_unit, axis=1)

In [28]:
toffee_df = ingest_excel(base_df, pd.read_excel('Cookie Ingredients.xlsx', sheet_name='Toffee'))

In [42]:
thin_and_crispy_df = ingest_excel(base_df, pd.read_excel('Cookie Ingredients.xlsx', sheet_name='Thin&Crispy'))
brown_butter_snickerdoodle_df = ingest_excel(base_df, pd.read_excel('Cookie Ingredients.xlsx', sheet_name='Brown Butter Muscovado Snickerd'))
cold_brew_df = ingest_excel(base_df, pd.read_excel('Cookie Ingredients.xlsx', sheet_name='Cold Brew Cookie w PB Chips'))

In [43]:
thin_and_crispy_df

,Brand,Ingredient,Weight/Amount_base,Cost,Per_Unit_Price,Weight/Amount_recipe,Unnamed: 3,Unnamed: 4
0,store brand,ap flour,5440 g,4.99,0.001,340g,NaN,NaN
1,king arthur,bread flour,2270 g,5.19,0.002,68g,NaN,NaN
2,store brand,white sugar,11330 g,13.59,0.001,288g,NaN,NaN
3,domino,dark brown sugar,453g,1.79,0.004,105g,NaN,NaN
4,morton's,coarse kosher salt,453g,1.68,0.004,2g,NaN,NaN
5,morton's,sea salt,737 g,2.99,0.004,5g,NaN,NaN
6,store brand,baking powder,284 g,1.99,0.007,5g,NaN,NaN
7,store brand,baking soda,454 g,0.89,0.002,5g,NaN,NaN
8,store brand,large eggs,60,6.99,0.117,2,NaN,NaN
9,mccormick's/name brand,vanilla,354 mL,29.99,0.085,15 mL,NaN,NaN


In [40]:
# calculate the cost of each ingredient in the recipe
def units_required_for_desired_yield(df, column, cookie):
    # using a list to hold the yield difference since apply() requires add'l args to be iterable.
    # In other words, I can't pass the Cookie or its float value for yield difference
    display(df)
    df[column] = df.apply(calculate_cost_for_desired_yield, args=([cookie.yield_difference]), axis=1)
    display(df)
    return df

ingredient_cost_column = 'Units_Required_For_Deisred_Yield'

In [38]:
toffee_df = units_required_for_desired_yield(toffee_df, ingredient_cost_column, toffee)

         Brand         Ingredient Weight/Amount_base  Cost  Per_Unit_Price  \
0       domino  light brown sugar             1810 g  4.49           0.002   
1     morton's           sea salt              737 g  2.99           0.004   
2  store brand    unsalted butter          24 sticks  7.49           0.312   

  Weight/Amount_recipe  Units_Required_For_Deisred_Yield  
0                 200g                             200.0  
1                   5g                               5.0  
2                    1                               1.0  
         Brand         Ingredient Weight/Amount_base  Cost  Per_Unit_Price  \
0       domino  light brown sugar             1810 g  4.49           0.002   
1     morton's           sea salt              737 g  2.99           0.004   
2  store brand    unsalted butter          24 sticks  7.49           0.312   

  Weight/Amount_recipe  Units_Required_For_Deisred_Yield  
0                 200g                             200.0  
1                   

In [33]:
toffee_df

,Brand,Ingredient,Weight/Amount_base,Cost,Per_Unit_Price,Weight/Amount_recipe,Units_Required_For_Deisred_Yield
0,domino,light brown sugar,1810 g,4.49,0.002,200g,200.0
1,morton's,sea salt,737 g,2.99,0.004,5g,5.0
2,store brand,unsalted butter,24 sticks,7.49,0.312,1,1.0


In [41]:
thin_and_crispy_df = units_required_for_desired_yield(thin_and_crispy_df, ingredient_cost_column, thin_and_crispy_cookie)

,Brand,Ingredient,Weight/Amount_base,Cost,Per_Unit_Price,Weight/Amount_recipe,Unnamed: 3,Unnamed: 4,Units_Required_For_Deisred_Yield
0,store brand,ap flour,5440 g,4.99,0.001,340g,NaN,NaN,68.0
1,king arthur,bread flour,2270 g,5.19,0.002,68g,NaN,NaN,13.6
2,store brand,white sugar,11330 g,13.59,0.001,288g,NaN,NaN,57.6
3,domino,dark brown sugar,453g,1.79,0.004,105g,NaN,NaN,21.0
4,morton's,coarse kosher salt,453g,1.68,0.004,2g,NaN,NaN,0.4
5,morton's,sea salt,737 g,2.99,0.004,5g,NaN,NaN,1.0
6,store brand,baking powder,284 g,1.99,0.007,5g,NaN,NaN,1.0
7,store brand,baking soda,454 g,0.89,0.002,5g,NaN,NaN,1.0
8,store brand,large eggs,60,6.99,0.117,2,NaN,NaN,0.4
9,mccormick's/name brand,vanilla,354 mL,29.99,0.085,15 mL,NaN,NaN,3.0


,Brand,Ingredient,Weight/Amount_base,Cost,Per_Unit_Price,Weight/Amount_recipe,Unnamed: 3,Unnamed: 4,Units_Required_For_Deisred_Yield
0,store brand,ap flour,5440 g,4.99,0.001,340g,NaN,NaN,68.0
1,king arthur,bread flour,2270 g,5.19,0.002,68g,NaN,NaN,13.6
2,store brand,white sugar,11330 g,13.59,0.001,288g,NaN,NaN,57.6
3,domino,dark brown sugar,453g,1.79,0.004,105g,NaN,NaN,21.0
4,morton's,coarse kosher salt,453g,1.68,0.004,2g,NaN,NaN,0.4
5,morton's,sea salt,737 g,2.99,0.004,5g,NaN,NaN,1.0
6,store brand,baking powder,284 g,1.99,0.007,5g,NaN,NaN,1.0
7,store brand,baking soda,454 g,0.89,0.002,5g,NaN,NaN,1.0
8,store brand,large eggs,60,6.99,0.117,2,NaN,NaN,0.4
9,mccormick's/name brand,vanilla,354 mL,29.99,0.085,15 mL,NaN,NaN,3.0


In [35]:
brown_butter_snickerdoodle_df = units_required_for_desired_yield(brown_butter_snickerdoodle_df, ingredient_cost_column, brown_snickerdoodle_cookie)
cold_brew_df = units_required_for_desired_yield(cold_brew_df, ingredient_cost_column, cold_brew_cookie)

In [36]:
thin_and_crispy_df

,Brand,Ingredient,Weight/Amount_base,Cost,Per_Unit_Price,Weight/Amount_recipe,Unnamed: 3,Unnamed: 4,Units_Required_For_Deisred_Yield
0,store brand,ap flour,5440 g,4.99,0.001,340g,NaN,NaN,68.0
1,king arthur,bread flour,2270 g,5.19,0.002,68g,NaN,NaN,13.6
2,store brand,white sugar,11330 g,13.59,0.001,288g,NaN,NaN,57.6
3,domino,dark brown sugar,453g,1.79,0.004,105g,NaN,NaN,21.0
4,morton's,coarse kosher salt,453g,1.68,0.004,2g,NaN,NaN,0.4
5,morton's,sea salt,737 g,2.99,0.004,5g,NaN,NaN,1.0
6,store brand,baking powder,284 g,1.99,0.007,5g,NaN,NaN,1.0
7,store brand,baking soda,454 g,0.89,0.002,5g,NaN,NaN,1.0
8,store brand,large eggs,60,6.99,0.117,2,NaN,NaN,0.4
9,mccormick's/name brand,vanilla,354 mL,29.99,0.085,15 mL,NaN,NaN,3.0


In [ ]:
# calcuate the sum of ingredients of the recipe (cost of the batch)
def cost_per_batch(df, column):
    cost = df[column].sum(axis=0)
    return cost

thin_crispy_per_batch = cost_per_batch(thin_and_crispy_df, cost_column)
brown_snickerdoodle_per_batch = cost_per_batch(brown_butter_snickerdoodle_df, cost_column)
cold_brew_per_batch = cost_per_batch(cold_brew_df, cost_column)

In [ ]:
def price_per_cookie(number_of_cookies, cost_per_batch):
    return round(cost_per_batch/number_of_cookies, ndigits=2)

price_per_thin_and_crispy = price_per_cookie(44, thin_crispy_per_batch)
price_per_brown_snickerdoodle = price_per_cookie(26, brown_snickerdoodle_per_batch)

In [ ]:
price_per_thin_and_crispy

In [ ]:
wage = 13  # $ per hour
labor_rate = wage/60  
total_production_time = 78
labor_cost_per_batch = round(labor_rate*total_production_time,2)
labor_cost_per_cookie = round(labor_cost_per_batch/thin_and_crispy_yield,2)

In [ ]:
cost_for_bags = 8.99
number_of_bags = 400
bag_cost_per_cookie = cost_for_bags/number_of_bags

In [ ]:
126*3